<a href="https://colab.research.google.com/github/pipeberrio/Analitica/blob/main/Tarea3_Anal%C3%ADtica_de_Negocios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tarea #3**


Raul Felipe Berrio Gomez CC *1035416370*

In [ ]:
import pandas as pd

# ---------------------------------
# 1. Descargar el archivo desde Google Drive y cargarlo
# ---------------------------------

# Descargar el archivo desde Google Drive usando el ID del archivo
# Nota: Asegúrate de que el archivo esté compartido públicamente para poder descargarlo
!gdown 110nTRpCHddZGW-dXA2d6M645IjFPCaMG

# Cargar el archivo en un DataFrame de pandas desde el archivo descargado
df = pd.read_excel('datos_pqr.xlsx')

# Renombrar las columnas del DataFrame para facilitar el manejo
df = df.rename(columns={
    'Radicado':'radicado',
    'No. codificación de registro de inscripción ':'registro de inscripción',
    'Mes/Año inscripción':'fecha',
    'Autoridad Ambiental Competente':'jurisdiccion',
    'Nombre o razon social':'razon social',
    'Número de identificación o NIT':'NIT',
    'Representante legal':'representante',
    'Dirección, física de la planta de transformación':'direccion',
    'Teléfono':'telefono',
    'Correo electrónico':'correo',
    'Capacidad instalada de transformación (ton/año)':'capacidad',
    'Tipo de material a transformar':'material',
    'Tipo de transformación':'transformacion',
    'No. del acto adminitrativo  por la cual se otorga permisos, concesiones y autorizaciónes del transformador  (si aplica)': 'No_acto',
    'Fecha del acto administrativo (si aplica)':'fecha_acto',
    'Descripción de la actividad autorizada mediante permisos, concesiones y autorizaciónes del transformador  (si aplica)': 'descripcion_act',
    'Observaciones': 'observaciones',
    'Responsable reporte':'responsable_reporte'
})

# ---------------------------------
# 2. Gestionar datos faltantes y duplicados
# ---------------------------------

# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Rellenar valores faltantes en las columnas 'telefono' y 'correo'
df['telefono'] = df['telefono'].fillna('No disponible')
df['correo'] = df['correo'].fillna('No disponible')

# Para columnas críticas como 'direccion', 'razon social', 'material' y 'transformacion', eliminamos las filas faltantes
df.dropna(subset=['direccion', 'razon social', 'material', 'transformacion'], inplace=True)

# ---------------------------------
# 3. Normalización de la columna 'direccion'
# ---------------------------------

# Eliminar espacios adicionales, convertir a título (capitalizar cada palabra), y remover caracteres especiales innecesarios
df['direccion'] = df['direccion'].str.strip().str.title()

# Reemplazar múltiples espacios por uno solo
df['direccion'] = df['direccion'].str.replace(' +', ' ', regex=True)

# Remover caracteres especiales innecesarios de la columna 'direccion'
df['direccion'] = df['direccion'].str.replace('[^A-Za-z0-9 ]', '', regex=True)

# ---------------------------------
# 4. Ajustar las columnas 'material' y 'transformacion'
# ---------------------------------

# Normalizar la columna 'material' para que esté en formato claro y limpio
df['material'] = df['material'].str.lower().str.strip()

# Si hay materiales múltiples, asegurarse de que estén separados correctamente (si están en una sola celda)
df['material'] = df['material'].str.replace(',', ';')  # Por ejemplo, separa por punto y coma si los materiales están juntos por coma

# Normalización de la columna 'transformacion'
df['transformacion'] = df['transformacion'].str.lower().str.strip()

# ---------------------------------
# 5. Manejo de outliers en 'capacidad'
# ---------------------------------

# Convertir la columna 'capacidad' a numérico (asegúrate de que no haya datos erróneos)
df['capacidad'] = pd.to_numeric(df['capacidad'], errors='coerce')

# Eliminar filas con valores de capacidad faltantes o valores extremos si es necesario
df.dropna(subset=['capacidad'], inplace=True)

# Calcular el rango intercuartil (IQR) para detectar y eliminar outliers en la capacidad
Q1 = df['capacidad'].quantile(0.25)
Q3 = df['capacidad'].quantile(0.75)
IQR = Q3 - Q1

# Filtrar los outliers fuera de 1.5 veces el rango intercuartil
df = df[(df['capacidad'] >= (Q1 - 1.5 * IQR)) & (df['capacidad'] <= (Q3 + 1.5 * IQR))]

# Mostrar los primeros registros del DataFrame limpio
df.head()

